In [1]:
import torch
import os
import json
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)
from platform import python_version
print(python_version())
from collections import defaultdict
from typing import Any, Iterable, List, Optional, Tuple, Union
from torch import Tensor
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_auc_score

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv
from torch_geometric.nn import NNConv
from torch_geometric.nn import global_mean_pool
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

1.13.1+cu116
3.8.10


In [2]:
# here is the modified to_networkx function that doesn't throw exceptions

def from_networkx(
    G: Any,
    group_node_attrs: Optional[Union[List[str], all]] = None,
    group_edge_attrs: Optional[Union[List[str], all]] = None,
) -> 'torch_geometric.data.Data':
    r"""Converts a :obj:`networkx.Graph` or :obj:`networkx.DiGraph` to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        G (networkx.Graph or networkx.DiGraph): A networkx graph.
        group_node_attrs (List[str] or all, optional): The node attributes to
            be concatenated and added to :obj:`data.x`. (default: :obj:`None`)
        group_edge_attrs (List[str] or all, optional): The edge attributes to
            be concatenated and added to :obj:`data.edge_attr`.
            (default: :obj:`None`)

    .. note::

        All :attr:`group_node_attrs` and :attr:`group_edge_attrs` values must
        be numeric.

    Examples:

        >>> edge_index = torch.tensor([
        ...     [0, 1, 1, 2, 2, 3],
        ...     [1, 0, 2, 1, 3, 2],
        ... ])
        >>> data = Data(edge_index=edge_index, num_nodes=4)
        >>> g = to_networkx(data)
        >>> # A `Data` object is returned
        >>> from_networkx(g)
        Data(edge_index=[2, 6], num_nodes=4)
    """
    import networkx as nx

    from torch_geometric.data import Data

    G = nx.convert_node_labels_to_integers(G)
    G = G.to_directed() if not nx.is_directed(G) else G

    if isinstance(G, (nx.MultiGraph, nx.MultiDiGraph)):
        edges = list(G.edges(keys=False))
    else:
        edges = list(G.edges)

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    data = defaultdict(list)

    if G.number_of_nodes() > 0:
        node_attrs = list(next(iter(G.nodes(data=True)))[-1].keys())
    else:
        node_attrs = {}

    if G.number_of_edges() > 0:
        edge_attrs = list(next(iter(G.edges(data=True)))[-1].keys())
    else:
        edge_attrs = {}

    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        if set(feat_dict.keys()) != set(node_attrs):
            raise ValueError('Not all nodes contain the same attributes')
        for key, value in feat_dict.items():
            data[str(key)].append(value)

    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        if set(feat_dict.keys()) != set(edge_attrs):
            raise ValueError('Not all edges contain the same attributes')
        for key, value in feat_dict.items():
            key = f'edge_{key}' if key in node_attrs else key
            data[str(key)].append(value)

    for key, value in G.graph.items():
        key = f'graph_{key}' if key in node_attrs else key
        data[str(key)] = value

    for key, value in data.items():
        if isinstance(value, (tuple, list)) and isinstance(value[0], Tensor):
            data[key] = torch.stack(value, dim=0)
        else:
            try:
                data[key] = torch.tensor(value)
            except:
                pass

    data['edge_index'] = edge_index.view(2, -1)
    data = Data.from_dict(data)

    if group_node_attrs is all:
        group_node_attrs = list(node_attrs)
    if group_node_attrs is not None:
        xs = []
        for key in group_node_attrs:
            x = data[key]
            x = x.view(-1, 1) if x.dim() <= 1 else x
            xs.append(x)
            del data[key]
        data.x = torch.cat(xs, dim=-1)

    if group_edge_attrs is all:
        group_edge_attrs = list(edge_attrs)
    if group_edge_attrs is not None:
        xs = []
        for key in group_edge_attrs:
            key = f'edge_{key}' if key in node_attrs else key
            x = data[key]
            x = x.view(-1, 1) if x.dim() <= 1 else x
            xs.append(x)
            del data[key]
        data.edge_attr = torch.cat(xs, dim=-1)

    if data.x is None and data.pos is None:
        data.num_nodes = G.number_of_nodes()

    return data

In [3]:
from torch_geometric.typing import SparseTensor

def to_edge_index(adj: Union[Tensor, SparseTensor]) -> Tuple[Tensor, Tensor]:
    r"""Converts a :class:`torch.sparse.Tensor` or a
    :class:`torch_sparse.SparseTensor` to edge indices and edge attributes.

    Args:
        adj (torch.sparse.Tensor or SparseTensor): The adjacency matrix.

    :rtype: (:class:`LongTensor`, :class:`Tensor`)

    Example:

        >>> edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
        ...                            [1, 0, 2, 1, 3, 2]])
        >>> adj = to_torch_coo_tensor(edge_index)
        >>> to_edge_index(adj)
        (tensor([[0, 1, 1, 2, 2, 3],
                [1, 0, 2, 1, 3, 2]]),
        tensor([1., 1., 1., 1., 1., 1.]))
    """
    if isinstance(adj, SparseTensor):
        row, col, value = adj.coo()
        if value is None:
            value = torch.ones(row.size(0), device=row.device)
        return torch.stack([row, col], dim=0), value

    if adj.requires_grad:
        # Calling adj._values() will return a detached tensor.
        # Use `adj.coalesce().values()` instead to track gradients.
        adj = adj.coalesce()
        return adj.indices(), adj.values()

    return adj._indices(), adj._values()

In [21]:
label_dict = {
        "Point": 0,
        "Line": 1,
        "Circle": 2,
        "Ellipse": 3,
        "Spline": 4,
        "Conic": 5,
        "Arc": 6,
        "External": 7,
        "Stop": 8,
        "Unknown": 9,
        "SN_Start": 11,
        "SN_End": 12,
        "SN_Center": 13
    }

edge_dict = {
    "Coincident": 0,
    "Projected": 1,
    "Mirror": 2,
    "Distance": 3,
    "Horizontal": 4,
    "Parallel": 5,
    "Vertical": 6,
    "Tangent": 7,
    "Length": 8,
    "Perpendicular": 9,
    "Midpoint": 10,
    "Equal": 11,
    "Diameter": 12,
    "Offset": 13,
    "Radius": 14,
    "Concentric": 15,
    "Fix": 16,
    "Angle": 17,
    "Circular_Pattern": 18,
    "Pierce": 19,
    "Linear_Pattern": 20,
    "Centerline_Dimension": 21,
    "Intersected": 22,
    "Silhoutted": 23,
    "Quadrant": 24,
    "Normal": 25,
    "Minor_Diameter": 26,
    "Major_Diameter": 27,
    "Rho": 28,
    "Unknown": 29,
    "Subnode": 30
}

In [22]:
from torch_geometric.utils import degree
def get_sketch_features(graph, feature_dim):
    x = torch.zeros([graph.num_nodes, feature_dim])

    
    
    for idx, p in enumerate(graph.parameters):
        
        # add one hot encoding to feature vector for node label
        #onePos = label_dict[graph.label[idx]]/7
        #for i in range(0, 14):
        #    x[idx, i] = 1 if onePos==i else 0
        x[idx, label_dict[graph.label[idx]]] = 1
        # convert label text into a feature value
        #x[idx, 14] = label_dict[graph.label[idx]]/7
        
        param_dict = json.loads(p)
        for i, k in enumerate(param_dict.keys()):
            
            if i+2 == feature_dim:
                break
            
            # convert each parameter value into a feature value
            x[idx, i+15] = float(param_dict[k])
        
        x[idx, -1] = degree(graph.edge_index[0], graph.num_nodes)[idx]
        #print(idx, p)
        #print(x[idx])
    return x

In [23]:
def get_sketch_attr_y(graph):
    y = torch.zeros([graph.num_nodes, 1], dtype=torch.int64)
    #rint(graph.label)
    
    
    for i, l in enumerate(graph.label):
        y[i, 0] = label_dict[l]
    
    return y

In [24]:
def get_sketch_adj(graph):
    tst = T.ToSparseTensor()
    return tst(graph).adj_t

In [25]:
def find_large_graph(graphs):
    bestN = 0
    bestI = 0
    for i, g in enumerate(graphs):
        if len(g) > bestN:
            bestN = len(g)
            bestI = i
    #print(bestI)
    return graphs[bestI]

In [26]:
def get_sketch_edge_attr(graph):
    dim = 31
    edge_attr = torch.zeros([len(graph.edge_label), dim])
    for idx, l in enumerate(graph.edge_label):
        edge_attr[idx, edge_dict[l]] = 1
    return edge_attr

In [27]:
# for now we will do node class sequence prediction

def get_sketch_construction_node_sequence(sg, graph):
    edge_sequence_indices = []
    node_sequence_indices = []
    node_sequence = torch.zeros((0, 15), dtype=torch.float) # 15th element is stop sign
    edge_sequence = torch.zeros((0, 32), dtype=torch.float) # 32nd element is stop sign
    start_node_idx=0
    start_edge_idx=0
    node_idx = 0
    edge_idx=0
    is_edge_sequence = False
    for elem in sg:
        
        if node_idx == len(graph.label):
            break
            
        if type(elem) == sketchgraphs.data.sequence.NodeOp:
            if is_edge_sequence:
                is_edge_sequence=False
                stop_token = torch.zeros((1, 32))
                stop_token[0, 31] = 1
                edge_sequence = torch.cat((edge_sequence, stop_token))
                edge_sequence_indices.append((start_edge_idx, edge_idx))
                start_node_idx=node_idx
                edge_idx+=1
            one_hot = torch.zeros((1, 15))
            one_hot[0, label_dict[graph.label[node_idx]]] = 1
            node_sequence = torch.cat((node_sequence, one_hot))
            node_idx+=1
        elif type(elem) == sketchgraphs.data.sequence.EdgeOp:
            #print(elem)
            if not is_edge_sequence:
                is_edge_sequence=True
                start_edge_idx=edge_idx
                stop_token = torch.zeros((1, 15))
                stop_token[0, 14] = 1
                node_sequence = torch.cat((node_sequence, stop_token))
                node_sequence_indices.append((start_node_idx, node_idx))
                node_idx+=1
            constraintNumber = edge_dict[graph.edge_label[edge_idx]]
            one_hot = torch.zeros((1, 32))
            one_hot[0, constraintNumber] = 1
            edge_sequence = torch.cat((edge_sequence, one_hot))
            edge_idx+=1
            
    #node_sequence[-1, 14] = 1
    #edge_sequence[-1, 31] = 1
    #print(node_sequence_indices, edge_sequence_indices)
    return node_sequence, edge_sequence, node_sequence_indices, edge_sequence_indices
        

In [28]:
import numpy as np
def get_sketchgraph_node_constraint_sequence(sg, graph):
    #y = torch.zeros((len(sg)-1, 2+31)) # 2 classes (node or edge), 14 node types (but contained in 31 positions for edge types), 31 edge types
    #node_idx=0
    #edge_idx=0
    #for idx, elem in enumerate(sg):
    #    if type(elem) == sketchgraphs.data.sequence.NodeOp and not elem.label == sketchgraphs.data._entity.EntityType.Stop:
    #        y[idx, 0] = 1
    #        y[idx, 2+label_dict[graph.label[node_idx]]] = 1  # 0.5 because we're marking 2 spaces in a vector of zeroes, to sum up to 1
    #        node_idx+=1
    #    elif type(elem) == sketchgraphs.data.sequence.EdgeOp:
    #        y[idx, 1] = 1
    #        y[idx, 2+edge_dict[graph.edge_label[edge_idx]]] = 1
    #        edge_idx+=1
    #return y
    y = torch.zeros((len(sg)-1), dtype=torch.long)
    node_idx=0
    edge_idx=0
    for idx, elem in enumerate(sg):
        if type(elem) == sketchgraphs.data.sequence.NodeOp and not elem.label == sketchgraphs.data._entity.EntityType.Stop:
            y[idx] = label_dict[graph.label[node_idx]]  # 0.5 because we're marking 2 spaces in a vector of zeroes, to sum up to 1
            node_idx+=1
        elif type(elem) == sketchgraphs.data.sequence.EdgeOp:
            y[idx] = 15+edge_dict[graph.edge_label[edge_idx]]
            edge_idx+=1
    return y


def convert_sketchgraph_to_pytorch(sg):
    # convert first to pyGraphViz graph using sketchgraph's function
    pgv_graph = sketchgraphs.data.sequence.pgvgraph_from_sequence(sg)
    # then to networkx graph
    nx_graph = nx.Graph(pgv_graph)
    # finally to pyTorch graph
    graph = from_networkx(nx_graph)
    return graph

def assign_attributes_to_graph(graph):
    graph.x = get_sketch_features(graph, 30)
    graph.y = get_sketch_attr_y(graph)
    graph.adj_t = get_sketch_adj(graph)
    graph.edge_index = to_edge_index(graph.adj_t)[0]
    graph.edge_attr = get_sketch_edge_attr(graph)
    return graph

def get_sketchgraph_graph_sequence(sg):
    generated_graph = []
    sequence = []
    for elem in sg:
        generated_graph.append(elem)
        new_graph = convert_sketchgraph_to_pytorch(generated_graph)
        if not hasattr(new_graph, 'edge_label'):
            new_graph.edge_label = []
        new_graph = assign_attributes_to_graph(new_graph)
        sequence.append(new_graph)
    return sequence

In [29]:
# custom dataset class of custom attributes
from torch_geometric.data import Dataset
class SketchgraphDataset(Dataset):
    def __init__(self, start_idx, end_idx, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(transform, pre_transform, pre_filter)
        
        self.data = []
        seq_data = flat_array.load_dictionary_flat('datasets/sg_t16_validation.npy')
        print(len(seq_data['sequences']))

        #test_graph_seq = find_large_graph(seq_data['sequences'])
        test_graph_seq = seq_data['sequences'][206778]
        test_graph_seq1 = seq_data['sequences'][10]

        sketchgraps_list = seq_data['sequences'][start_idx:end_idx]

        #link_transform = T.RandomLinkSplit(is_undirected=True, key="edge_attr")
        
        for sg in sketchgraps_list:
            #print(test_graph_seq)
            graph = convert_sketchgraph_to_pytorch(sg)
            
            if not hasattr(graph, 'edge_label'):
                continue

            # next we need to add required attributes: x, y, adj_t
            graph.x = get_sketch_features(graph, 30)
            graph.y = get_sketch_attr_y(graph)
            graph.adj_t = get_sketch_adj(graph)
            graph.edge_index = to_edge_index(graph.adj_t)[0]
            graph.edge_attr = get_sketch_edge_attr(graph)
            graph.seq = get_sketchgraph_graph_sequence(sg)
            graph.seq_y = get_sketchgraph_node_constraint_sequence(sg, graph)
            #graph.node_sequence, graph.edge_sequence, graph.node_sequence_indices, graph.edge_sequence_indices = get_sketch_construction_node_sequence(sg, graph)
            #print(len(graph.edge_label))
            #graph = link_transform(graph)
            self.data.append(graph)
            
    def len(self):
        return len(self.data)
    
    def get(self, idx):
        return self.data[idx]
    
"""def custom_collate(batch, b):
    print("AAAAA")
    edge_label_batch = []
    for b in batch:
        #print(b.edge_label)
        edge_label_batch.append(b.edge_label)
    return edge_label_batch"""
    

'def custom_collate(batch, b):\n    print("AAAAA")\n    edge_label_batch = []\n    for b in batch:\n        #print(b.edge_label)\n        edge_label_batch.append(b.edge_label)\n    return edge_label_batch'

In [30]:
import sketchgraphs
import networkx as nx
from sketchgraphs.data import flat_array

train_dataset = SketchgraphDataset(0, 300)
test_dataset = SketchgraphDataset(101, 111)
    
data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

next(iter(data_loader)).seq_y
#for b in iter(data_loader):
#    print(b)
#print(graph)
#print(graph.x)
#print(graph.y)

315228
315228


tensor([ 7,  1, 11, 15, 12, 45,  1, 45, 20, 11, 15, 12, 45,  0, 15,  1, 11, 45,
        15, 12, 20,  1, 19, 45, 11, 45, 12, 15,  1, 45, 11, 15, 45, 12, 15, 15,
         1, 45, 11, 45, 20, 15, 12, 45, 15,  1, 11, 45, 15, 12, 20,  1, 19, 45,
        11, 45, 12, 24,  1, 45, 11, 15, 45, 12, 15, 24,  1, 45, 11, 45, 20, 15,
        12, 45, 15,  0, 45, 20,  0, 45, 45])

In [42]:
#print(next(iter(data_loader)).x)
val_dataset = SketchgraphDataset(500, 700)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

315228


In [ ]:
class linkPredictionGAE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(linkPredictionGAE, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.conv1 = NNConv(input_dim, hidden_dim, nnconvnn(input_dim, hidden_dim))
        self.conv2 = NNConv(hidden_dim, output_dim, nnconvnn(hidden_dim, output_dim))
        
    def encode(self, x, pos_edge_index, edge_attr):
        x = self.conv1(x, pos_edge_index, edge_attr)
        x = x.relu()
        return self.conv2(x, pos_edge_index, edge_attr)
    
    def decode(self, z, pos_edge_index, neg_edge_index):
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)
        return logits
    
    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [17]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    dataset_name = 'ogbn-arxiv'
    dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())
    data = dataset[0]
    print(data)
    #data = batch
    


    # Make the adjacency matrix to symmetric
    data.adj_t = data.adj_t.to_symmetric()
    #print(data.y)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    #device = 'cpu'
    # If you use GPU, the device should be cuda
    print('Device: {}'.format(device))
    data = data.to(device)
    split_idx = dataset.get_idx_split()
    print(split_idx['train'])
    #train_idx = torch.LongTensor(range(0, data.num_nodes)).to(device)
    #print(train_idx)

Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])
Device: cuda
tensor([     0,      1,      2,  ..., 169145, 169148, 169251])


In [16]:
def get_link_labels(pos_edge_index, neg_edge_index):
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1
    return link_labels

In [17]:
def train_link_predictor(model):
    model.train()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    loss_avg = 0
    batch_count = 0
    for batch in iter(data_loader):
        batch = batch.to(device)
        batch_count+=1
        neg_edge_index = negative_sampling(
            edge_index=batch.edge_index,
            num_nodes=batch.num_nodes,
            num_neg_samples=batch.edge_index.size(1)
        )
        
        optimizer.zero_grad()
        
        z = model.encode(batch.x, batch.edge_index, batch.edge_attr)
        link_logits = model.decode(z, batch.edge_index, neg_edge_index)
        link_labels = get_link_labels(batch.edge_index, neg_edge_index)
        loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
        loss.backward()
        optimizer.step()
        loss_avg += loss.item()
        
    return loss_avg / batch_count

In [437]:
def test(model):
    model.eval()
    perf=0
    batch_count = 0
    for batch in iter(test_loader):
        batch = batch.to(device)
        neg_edge_index = negative_sampling(
            edge_index=batch.edge_index,
            num_nodes=batch.num_nodes,
            num_neg_samples=batch.edge_index.size(1)
        )
        
        z = model.encode(batch.x, batch.edge_index, batch.edge_attr)
        link_logits = model.decode(z, batch.edge_index, neg_edge_index)
        link_probs = link_logits.sigmoid()
        link_labels = get_link_labels(batch.edge_index, neg_edge_index)
        try:
            perf+=roc_auc_score(link_labels.cpu().detach().numpy(), link_probs.cpu().detach().numpy())
            batch_count+=1
        except:
            pass
        
    return perf/batch_count

In [90]:
# training link prediction model

model = linkPredictionGAE(30, 128, 64).to(device)

epochs = 5000
for e in range(epochs):
    train_loss = train_link_predictor(model)
    perf = test(model)
    print(train_loss, perf)

NameError: name 'linkPredictionGAE' is not defined

In [431]:
testGraph = next(iter(test_loader)).to(device)
z = model.encode(testGraph.x, testGraph.edge_index, testGraph.edge_attr)
final_edge_index = model.decode_all(z)
print(final_edge_index)
print(testGraph.edge_index)

tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
         4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7,
         7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9],
        [0, 1, 2, 5, 6, 7, 0, 1, 2, 5, 6, 7, 0, 1, 2, 4, 5, 6, 7, 9, 3, 4, 8, 9,
         2, 3, 4, 7, 8, 9, 0, 1, 2, 5, 6, 7, 0, 1, 2, 5, 6, 7, 0, 1, 2, 4, 5, 6,
         7, 9, 3, 4, 8, 9, 2, 3, 4, 7, 8, 9]], device='cuda:0')
tensor([[0, 1, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, 7, 8, 9, 9],
        [2, 2, 0, 1, 4, 4, 2, 3, 7, 7, 5, 6, 9, 9, 7, 8]], device='cuda:0')


In [18]:
class rnnModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_size, output_dim):
        super(rnnModel, self).__init__()
        self.layer_count = 2
        self.batch_size = 64
        self.hidden_size = hidden_size
        self.preprocess_gcn = NNConv(30, hidden_size, nnconvnn(30, hidden_size))
        #self.gcn_mlp = torch.nn.Linear(hidden_size, self.layer_count)
        self.rnn = torch.nn.RNN(input_dim, hidden_size, self.layer_count) # batch first = True means the first dimension is batch size
        self.mlp = torch.nn.Linear(hidden_size, output_dim)
        # x -> batch_size x sequence_length x input_dim if we wanted many to one RNN
        # for many to many, we use sequence length of 1
        self.hidden_state = torch.zeros((self.layer_count, self.hidden_size), dtype=torch.float)
        
        
    def reset_parameters(self):
        self.rnn.reset_parameters()
        self.mlp.reset_parameters()
        
    def forward(self, x, graph_x, edge_index, edge_attr, prev_hid=None):
        #print(self.rnn._flat_weights[0].dtype, x.dtype)
        #self.hidden_state.to(device)
        hidden=None
        if prev_hid==None:
            hidden = self.preprocess_gcn(graph_x, edge_index, edge_attr)
        #hidden = self.gcn_mlp(hidden)
            hidden = global_mean_pool(hidden, torch.zeros((graph_x.size(0)), dtype=torch.long))
            hidden = torch.cat((hidden, hidden))
        else:
            hidden=prev_hid
            
        x, self.hidden_state = self.rnn(x, hidden)
        x = self.mlp(x)
        torch.nn.LogSoftmax(dim=-1)
        x = F.softmax(x, dim=1)
        return x, self.hidden_state.detach()
    
    def generate(self, graph_x, ):
        pass
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros((self.layer_count, self.hidden_size), dtype=torch.float)
        return hidden
    

In [37]:
class nnconvnn(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(nnconvnn, self).__init__()
        
        self.simpleLin = torch.nn.Linear(31, input_dim*output_dim)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        self.simpleLin.reset_parameters()
        
    def forward(self, x):
        
        x = self.simpleLin(x)
        return x
        

In [38]:
class newrnnmodel(torch.nn.Module):
    def __init__(self, input_dim, hidden_size, output_dim_second, num_nnconv_layers):
        super(newrnnmodel, self).__init__()
        self.layer_count = 2
        self.hidden_size = hidden_size
        self.softmax = torch.nn.LogSoftmax(dim=-1)
        self.input_gcn = torch.nn.ModuleList()
        self.input_gcn.append(NNConv(30, hidden_size, nnconvnn(30, hidden_size)))
        #self.bns = torch.nn.ModuleList()
        for i in range(num_nnconv_layers-1):
            self.input_gcn.append(NNConv(hidden_size, hidden_size, nnconvnn(hidden_size, hidden_size)))
            
        #for i in range(num_nnconv_layers-1):
        #    self.bns.append(torch.nn.BatchNorm1d(hidden_size))

        self.rnn_class = torch.nn.RNN(hidden_size, hidden_size, self.layer_count) # batch first = True means the first dimension is batch size

        self.mlp_pre_rnn = torch.nn.Linear(hidden_size, hidden_size)
        self.mlp_class = torch.nn.Linear(hidden_size, output_dim_second)
        
    def reset_parameters(self):

        self.rnn_class.reset_parameters()
        self.mlp_class.reset_parameters()
        for l in range(len(self.input_gcn)):
            self.input_gcn[l].reset_parameters()
        
    def forward(self, graph_sequence):

        hidden = torch.zeros((self.layer_count, self.hidden_size), dtype=torch.float).to(device)
        F.dropout.training = self.training
        output = torch.tensor([]).to(device)
        for g in graph_sequence:
            x = g.x
            for l in range(len(self.input_gcn)):
                x = self.input_gcn[l](x, g.edge_index, g.edge_attr)
                x = F.relu(x)
                #x = F.dropout(x, p=0.2)
            x = global_mean_pool(x, torch.zeros((g.x.size(0)), dtype=torch.long).to(device))
            
            x = self.mlp_pre_rnn(x)
            
            x, hidden = self.rnn_class(x, hidden)
            
            x = self.mlp_class(x)
        
            x = self.softmax(x)
            
            output = torch.cat((output, x))
        return output, hidden.detach()
        
    def predict_next(self, sequence):
        pred, hidden = self.forward(sequence)
        return torch.argmax(pred[-1], dim=0)
        
    def init_hidden(self, batch_size):
        #hidden = torch.zeros((self.layer_count, self.hidden_size), dtype=torch.float)
        #return hidden
        pass

In [39]:
rnn = newrnnmodel(15, 16, 31+15, 2).to(device)

optimizer_class = torch.optim.Adam(rnn.parameters(), lr=0.01)
loss_fn = F.nll_loss
#loss_fn = F.mse_loss
epochs = 20
rnn.train()

for e in range(epochs):
    loss_avg = 0
    count = 0
    acc = 0
    for batch in iter(data_loader):
        count+=1
        batch.to(device)

        optimizer_class.zero_grad()
        pred_class, _ = rnn.forward(batch.seq[0])
        #print(pred_type)
        #print(pred_class)
        #print(batch.seq_y.size())
        loss_class = loss_fn(pred_class[:-1], batch.seq_y)
        pred_index = torch.argmax(pred_class[:-1], dim=1)
        acc += torch.sum(pred_index==batch.seq_y).item() / len(batch.seq_y)
        loss_class.backward()
        
        loss_avg += loss_class.item()
        

        optimizer_class.step()
        
        
    print(loss_avg/count, "Acc: " + str(acc/count * 100) + "%")


2.085012123138211 Acc: 39.17942178718779%
1.666133288157026 Acc: 48.20912219759601%
1.5573732196088619 Acc: 50.58895868659086%
1.48229676375421 Acc: 54.07953633750664%
1.1791081705121291 Acc: 64.15743618389408%
1.08879535682624 Acc: 66.72323824474009%
1.0097746913847716 Acc: 67.50647380471196%
0.9837427906766784 Acc: 69.00742232387098%
0.9742474609294464 Acc: 69.13851806216064%
0.9834740185069799 Acc: 68.35246355409387%
0.9798067321247081 Acc: 68.92727099616673%
0.974554391858171 Acc: 68.23420336620252%
0.9364402917117179 Acc: 69.64798486470889%
0.9377042466033263 Acc: 69.65832404277386%
1.0009042897252334 Acc: 67.95296438326463%
0.9970688350423921 Acc: 68.05789546986028%
0.9077694527181893 Acc: 69.67256946287162%
0.9400986725321183 Acc: 69.12506865858643%
0.8980874640736293 Acc: 70.45778186129382%
0.8798653557647431 Acc: 70.83856483605963%


In [41]:
rnn.eval()
acc = 0
count = 0
loss_avg = 0
for batch in iter(val_loader):
    count+=1
    batch.to(device)
    pred_class,_ = rnn.forward(batch.seq[0])
    loss_class = loss_fn(pred_class[:-1], batch.seq_y)
    pred_index = torch.argmax(pred_class[:-1], dim=1)
    acc += torch.sum(pred_index==batch.seq_y).item() / len(batch.seq_y)
    #print(loss_class.item())
    #print(F.softmax(pred_class, dim=-1)[:-1])
    #print(batch.seq_y)
    loss_avg += loss_class.item()
print(loss_avg/count, "Acc: " + str(acc/count * 100) + "%")

0.765283055754021 Acc: 73.83273358869177%


In [ ]:
# testing RNN
nodeModel = rnnModel(15, 8, 15)
edgeModel = rnnModel(32, 8, 32)
node_optimizer = torch.optim.Adam(nodeModel.parameters(), lr=0.01)
edge_optimizer = torch.optim.Adam(edgeModel.parameters(), lr=0.01)
loss_fn = F.binary_cross_entropy
for e in range(100):
    for batch in iter(data_loader):
        #batch = batch.to(device)
        #print(batch)
        edge_hidden_state = None
        node_hidden_state = None
        for idx in range(len(batch.edge_sequence_indices[0])):
            # first run node prediction model
            #print("Node predicition half")
            node_pair = batch.node_sequence_indices[0][idx]
            #print(node_pair)
            node_pred, node_hidden_state = nodeModel.forward(batch.node_sequence[node_pair[0]:node_pair[1]], batch.x, batch.edge_index, batch.edge_attr, edge_hidden_state)
            node_target = batch.node_sequence[node_pair[0]+1:node_pair[1]+1]
            
            #print("Edge predicition half")
            # then run edge prediction model
            edge_pair = batch.edge_sequence_indices[0][idx]
            #print(edge_pair)
            edge_pred, edge_hidden_state = edgeModel.forward(batch.edge_sequence[edge_pair[0]:edge_pair[1]], batch.x, batch.edge_index, batch.edge_attr)
            edge_target = batch.edge_sequence[edge_pair[0]+1:edge_pair[1]+1]
            
            loss = loss_fn(node_pred, node_target)
            node_optimizer.zero_grad()
            loss.backward()
            node_optimizer.step()
            print(loss.item())
            
            loss = loss_fn(edge_pred, edge_target)
            edge_optimizer.zero_grad()
            loss.backward()
            edge_optimizer.step()
            print(loss.item())
            
        #pred = nodeModel.forward(batch.node_sequence[0:-1])
        #loss = loss_fn(pred, batch.node_sequence[1:])
        
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
        

        


In [14]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # TODO: Implement a function that initializes self.convs, 
        # self.bns, and self.softmax.

        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = None

        # A list of 1D batch normalization layers
        self.bns = None

        # The log softmax layer
        self.softmax = None

        ############# Your code here ############
        ## Note:
        ## 1. You should use torch.nn.ModuleList for self.convs and self.bns
        ## 2. self.convs has num_layers GCNConv layers
        ## 3. self.bns has num_layers - 1 BatchNorm1d layers
        ## 4. You should use torch.nn.LogSoftmax for self.softmax
        ## 5. The parameters you can set for GCNConv include 'in_channels' and 
        ## 'out_channels'. For more information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 6. The only parameter you need to set for BatchNorm1d is 'num_features'
        ## For more information please refer to the documentation: 
        ## https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        ## (~10 lines of code)

        #self.testnnconv = 
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        self.softmax = torch.nn.LogSoftmax(dim=-1)

        for i in range(num_layers - 2):
            self.convs.append(NNConv(input_dim, hidden_dim, nnconvnn(input_dim, hidden_dim)))
        self.convs.append(NNConv(hidden_dim, hidden_dim, nnconvnn(hidden_dim, hidden_dim)))
        self.linear = torch.nn.Linear(hidden_dim, output_dim)
        for i in range(num_layers - 1):
            self.bns.append(torch.nn.BatchNorm1d(hidden_dim))
            
        
        #########################################

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        # TODO: Implement a function that takes the feature tensor x and
        # edge_index tensor adj_t and returns the output tensor as
        # shown in the figure.

        out = None

        ############# Your code here ############
        ## Note:
        ## 1. Construct the network as shown in the figure
        ## 2. torch.nn.functional.relu and torch.nn.functional.dropout are useful
        ## For more information please refer to the documentation:
        ## https://pytorch.org/docs/stable/nn.functional.html
        ## 3. Don't forget to set F.dropout training to self.training
        ## 4. If return_embeds is True, then skip the last softmax layer
        ## (~7 lines of code)
        F.dropout.training = self.training
        #x = self.testnnconv(x, edge_index, edge_attr)
        for i in range(len(self.convs) - 1):
            x = self.convs[i](x, edge_index, edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x)
        x = self.convs[-1](x, edge_index, edge_attr)
        x = self.linear(x)
        if self.return_embeds == False:
            x = self.softmax(x)
        out = x
        #########################################

        return out

In [15]:
def train(model, loader, train_idx, optimizer, loss_fn):
    # TODO: Implement a function that trains the model by 
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    ############# Your code here ############
    ## Note:
    ## 1. Zero grad the optimizer
    ## 2. Feed the data into the model
    ## 3. Slice the model output and label by train_idx
    ## 4. Feed the sliced output and label to loss_fn
    ## (~4 lines of code)
    for batch in iter(loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        o = model(batch.x, batch.edge_index, batch.edge_attr)
        # o = o[train_idx] # we train on the whole graphs now
        #print(o)
        #print(batch.y.squeeze())
        loss = loss_fn(o, batch.y.squeeze(1))
        #########################################

        loss.backward()
        optimizer.step()

    return loss.item()

In [16]:
# Test function here
@torch.no_grad()
def test(model, loader, split_idx, evaluator, save_model_results=False):
    # TODO: Implement a function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None
    train_acc = 0
    valid_acc = 0
    test_acc = 0
    
    count = 0
    
    for batch in loader:
        count+=1
        batch = batch.to(device)
        ############# Your code here ############
        ## (~1 line of code)
        ## Note:
        ## 1. No index slicing here
        out = model(batch.x, batch.edge_index, batch.edge_attr)
        #########################################

        y_pred = out.argmax(dim=-1, keepdim=True)
        
        #if count == 1:
            #print(y_pred, batch.y)
        
        train_acc += evaluator.eval({
            'y_true': batch.y,
            'y_pred': y_pred,
        })['acc']
        valid_acc += evaluator.eval({
            'y_true': batch.y,
            'y_pred': y_pred,
        })['acc']
        test_acc += evaluator.eval({
            'y_true': batch.y,
            'y_pred': y_pred,
        })['acc']

    train_acc /= count
    valid_acc /= count
    test_acc /= count

    if save_model_results:
        print ("Saving Model Predictions")

        data = {}
        data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

        df = pd.DataFrame(data=data)
        # Save locally as csv
        df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [17]:
# Please do not change the args
if 'IS_GRADESCOPE_ENV' not in os.environ:
    args = {
      'device': device,
      'num_layers': 3,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 100,
    }
    args

In [18]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    model = GCN(30, args['hidden_dim'],
              14, args['num_layers'],
              args['dropout']).to(device)
    evaluator = Evaluator(name='ogbn-arxiv')

In [19]:
# Please do not change these args
# Training should take <10min using GPU runtime
import copy
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # reset the parameters to initial random value
    model.reset_parameters()

    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
    loss_fn = F.nll_loss

    best_model = None
    best_valid_acc = 0

    for epoch in range(1, 1 + args["epochs"]):
        loss = train(model, data_loader, [], optimizer, loss_fn)
        result = test(model, test_loader, [], evaluator)
        train_acc, valid_acc, test_acc = result
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            best_model = copy.deepcopy(model)
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 1.1944, Train: 66.89%, Valid: 66.89% Test: 66.89%
Epoch: 02, Loss: 0.7170, Train: 72.64%, Valid: 72.64% Test: 72.64%
Epoch: 03, Loss: 0.6501, Train: 74.53%, Valid: 74.53% Test: 74.53%
Epoch: 04, Loss: 0.4657, Train: 75.57%, Valid: 75.57% Test: 75.57%
Epoch: 05, Loss: 0.4036, Train: 75.54%, Valid: 75.54% Test: 75.54%
Epoch: 06, Loss: 0.4142, Train: 75.61%, Valid: 75.61% Test: 75.61%
Epoch: 07, Loss: 0.3661, Train: 77.97%, Valid: 77.97% Test: 77.97%
Epoch: 08, Loss: 0.3721, Train: 79.85%, Valid: 79.85% Test: 79.85%
Epoch: 09, Loss: 0.3096, Train: 80.93%, Valid: 80.93% Test: 80.93%
Epoch: 10, Loss: 0.3449, Train: 80.03%, Valid: 80.03% Test: 80.03%
Epoch: 11, Loss: 0.3756, Train: 81.84%, Valid: 81.84% Test: 81.84%
Epoch: 12, Loss: 0.2886, Train: 79.02%, Valid: 79.02% Test: 79.02%
Epoch: 13, Loss: 0.3435, Train: 83.23%, Valid: 83.23% Test: 83.23%
Epoch: 14, Loss: 0.3259, Train: 85.24%, Valid: 85.24% Test: 85.24%
Epoch: 15, Loss: 0.2378, Train: 86.44%, Valid: 86.44% Test: 86

In [ ]:
result = test(model, data_loader, [], evaluator)